In [1]:
# !pip install -q efficientnet >> /dev/null

In [2]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from kaggle_datasets import KaggleDatasets
import matplotlib.pyplot as plt
import tqdm
import random
import tensorflow.keras.backend as K
from sklearn import preprocessing, model_selection
import tensorflow_addons as tfa
# import efficientnet.tfkeras as efn

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print(f'Running on TPU {tpu.master()}')
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()
    

REPLICAS = strategy.num_replicas_in_sync
print(f'REPLICAS: {REPLICAS}')


BATCH_SIZE = 16 * strategy.num_replicas_in_sync
VAL_SIZE = 10000
AUTO = tf.data.experimental.AUTOTUNE

GCS_PATH_train_p1 = KaggleDatasets().get_gcs_path('seti-tfrec-train-p1-6ch')
GCS_PATH_train_p2 = KaggleDatasets().get_gcs_path('seti-tfrec-train-p2-ch6')
GCS_PATH_train_p3 = KaggleDatasets().get_gcs_path('seti-tfrec-train-p3-6ch')
GCS_PATH_train_p4 = KaggleDatasets().get_gcs_path('seti-tfrec-train-p4-6ch')

GCS_PATH_test_p1 = KaggleDatasets().get_gcs_path('seti-tfrec-test11-p1')
GCS_PATH_test_p2 = KaggleDatasets().get_gcs_path('seti-tfrec-test11-p2')
GCS_PATH_test_p3 = KaggleDatasets().get_gcs_path('seti-tfrec-test11-p3')
GCS_PATH_test_p4 = KaggleDatasets().get_gcs_path('seti-tfrec-test11-p4')

Running on TPU grpc://10.0.0.2:8470
REPLICAS: 8


In [3]:
NUM_TRAIN_SAMPLES = 50165

In [4]:
def prepare_img(byte_img):
    img1 = tf.io.decode_raw(byte_img[0], out_type=tf.float32)
    img1 = tf.reshape(img1, (255, 255, 3))
    img2 = tf.io.decode_raw(byte_img[1], out_type=tf.float32)
    img2 = tf.reshape(img2, (255, 255, 3))
    img = tf.concat([img1, img2], axis=-1)
#     img = (img + 17.47) / (17.47 + 179.1)
#     img = tf.cast(img, tf.float32)
#     img = tf.image.resize(img, (255, 255))
#     img = tf.image.resize(img, (331, 331))
#     img = img / 100.
    return img

def read_tfrec(intance):
    tfrec_format = {
                    'image1' : tf.io.FixedLenFeature([], tf.string),
                    'image2' : tf.io.FixedLenFeature([], tf.string),
                    'image_id' : tf.io.FixedLenFeature([], tf.string),
                    'target' : tf.io.FixedLenFeature([], tf.int64)
                    }  
    tfrec = tf.io.parse_single_example(intance, tfrec_format)
    return [tfrec['image1'], tfrec['image2']], tfrec['target']
# tf.transpose(tfrec['image'], [2, 0, 1])
def read_tfrec_test(intance):
    tfrec_format = {
                    'image1' : tf.io.FixedLenFeature([], tf.string),
                    'image2' : tf.io.FixedLenFeature([], tf.string),
                    'image_id' : tf.io.FixedLenFeature([], tf.string),
                    }  
    tfrec = tf.io.parse_single_example(intance, tfrec_format)
    return [tfrec['image1'], tfrec['image2']], tfrec['image_id']

In [5]:
# imgs.values[0].shape

In [6]:
# tf.keras.layers.experimental.preprocessing.Normalization(axis=[1, 2])(imgs.values[0])

In [7]:
with strategy.scope():
    def create_model():
        base_model1 = tf.keras.applications.EfficientNetB1(include_top=False, weights='imagenet',input_shape=(255,255,3))
#         base_model1 = tf.keras.applications.NASNetLarge(include_top=False, weights='imagenet',input_shape=(255,255,3))
        for l in base_model1.layers:
            l._name = l.name
        base_model1._name = base_model1.name + '1'
#         base_model1.trainable = False
        inp = tf.keras.layers.Input(shape=(6, 255, 255))
#         input1_ = tf.keras.layers.experimental.preprocessing.Normalization(axis=[2, 3])(inp)
        inp1 = tf.keras.layers.Concatenate()([inp[..., tf.newaxis] for _ in range(3)])
        model_ = tf.keras.models.Sequential([base_model1,
                                             tf.keras.layers.GlobalAveragePooling2D(),
                                             tf.keras.layers.Dense(64),
                                             tf.keras.layers.BatchNormalization(),
                                             tf.keras.layers.ReLU()])
        x = tf.keras.layers.TimeDistributed(model_)(inp1)
        x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(512, return_sequences=True))(x)
        x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(512))(x)
        #     x = tf.keras.layers.Dropout(.3)(x)
        x = tf.keras.layers.Dense(32)(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.ReLU()(x)
        #     x = tf.keras.layers.Dropout(.3)(x)
        x = tf.keras.layers.Dense(1,activation='sigmoid')(x)
        model = tf.keras.Model(inputs=inp, outputs=x)
        return model

In [8]:
train_tfrec = np.array(tf.io.gfile.glob(GCS_PATH_train_p1 + '/train_tfrec/*') + 
                       tf.io.gfile.glob(GCS_PATH_train_p2 + '/train_tfrec/*') +
                       tf.io.gfile.glob(GCS_PATH_train_p3 + '/train_tfrec/*') +
                       tf.io.gfile.glob(GCS_PATH_train_p4 + '/train_tfrec/*'))
test_OOF = train_tfrec[0]
train_tfrec = train_tfrec[1:]
# random.shuffle(train_tfrec)
for seed in range(0, 1):
    skf = model_selection.KFold(n_splits=5, random_state=1370 + seed, shuffle=True)
    for fold, (it, iv) in enumerate(skf.split(train_tfrec)):

    #     tf.tpu.experimental.initialize_tpu_system(tpu)
        K.clear_session()
        # @tf.function
        def training_(imgs, lbls):
            imgs = tf.transpose(imgs, [0, 3, 1, 2])
            lbls = tf.cast(lbls, tf.float32)
            w_ = (1 + lbls * 15) / 16
            lbls = lbls[..., tf.newaxis]
            with tf.GradientTape() as tape:
                y_pred = model(imgs, training=True)
                losses = loss(lbls, y_pred, sample_weight=w_)
            grads = tape.gradient(losses, model.trainable_variables)
            opt.apply_gradients(zip(grads, model.trainable_variables))
            auc.update_state(lbls, y_pred)
            return tf.reduce_mean(losses)
        # @tf.function
        def validation_(imgs, lbls):
            lbls = tf.cast(lbls, tf.float32)
            imgs = tf.transpose(imgs, [0, 3, 1, 2])
            w_ = (1 + lbls * 15) / 16
            lbls = lbls[..., tf.newaxis]
            y_pred = model(imgs)
            auc.update_state(lbls, y_pred)
            loss_ = loss(lbls, y_pred, sample_weight=w_)
            loss_ = tf.reduce_mean(loss_)
            return loss_

        @tf.function
        def validatation_dist(imgs, lbls):
            per_rep_losses = strategy.run(validation_, args=(imgs, lbls,))
            dist_losses = strategy.reduce("MEAN", per_rep_losses, axis=None)
            return dist_losses

        @tf.function
        def training_dist(imgs, lbls):
            per_rep_losses = strategy.run(training_, args=(imgs, lbls,))
            dist_losses = strategy.reduce("MEAN", per_rep_losses, axis=None)
            return dist_losses
        ds_train = tf.data.TFRecordDataset(train_tfrec[it], num_parallel_reads=AUTO)
        ds_train = ds_train.shuffle(1024*50, seed=1370)
        opti = tf.data.Options()
        opti.experimental_deterministic = False
        ds_train = ds_train.with_options(opti)
        ds_train = ds_train.map(read_tfrec, num_parallel_calls=AUTO)
        ds_train = ds_train.map(lambda img, lbl: (prepare_img(img), lbl))


        ds_valid = tf.data.TFRecordDataset(train_tfrec[iv], num_parallel_reads=AUTO)
        ds_valid = ds_valid.map(read_tfrec, num_parallel_calls=AUTO)
        ds_valid = ds_valid.map(lambda img, lbl: (prepare_img(img), lbl))

        ds_train = ds_train.batch(BATCH_SIZE, drop_remainder=True).prefetch(AUTO)
        ds_train_dist = strategy.experimental_distribute_dataset(ds_train)
        ds_valid = ds_valid.batch(BATCH_SIZE, drop_remainder=True).prefetch(AUTO)
        ds_valid_dist = strategy.experimental_distribute_dataset(ds_valid)

        validation_res = 0

        with strategy.scope():
            opt = tf.keras.optimizers.Adam(learning_rate=0.001)
#             opt = tfa.optimizers.AdamW(
#                                 weight_decay=1e-7,
#                                 learning_rate=1e-4,
#                                 beta_1=0.9,
#                                 beta_2=0.999,
#                                 epsilon=1e-07,
#                                 amsgrad=True,
#                                 name='AdamW',
#                             )
        #     loss = tf.keras.losses.BinaryCrossentropy(label_smoothing=0.01, reduction='none')
            def loss(y_true, y_pred, sample_weight):
#                 return tf.multiply(tf.keras.losses.binary_crossentropy(y_true, y_pred), sample_weight)
                return tfa.losses.sigmoid_focal_crossentropy(y_true, y_pred)
        # #         return tf.nn.weighted_cross_entropy_with_logits(y_true, y_pred, 10)
            auc = tf.keras.metrics.AUC(num_thresholds=50)
            model = create_model()

        for epoch in range(20):
            print(f'Epoch {epoch} Fold {fold}:')
            losses = tf.Variable(0.)
            with strategy.scope():
    #             for imgs, lbls in ds_train_dist:
                for _, (imgs, lbls) in zip(tqdm.tqdm(range((NUM_TRAIN_SAMPLES - VAL_SIZE) // BATCH_SIZE), leave=True, position=0), ds_train_dist):
                    losses.assign_add(training_dist(imgs, lbls))
            print('loss_training: ',(losses / ((NUM_TRAIN_SAMPLES - VAL_SIZE) // BATCH_SIZE)).numpy())
            print('AUC_training: ', auc.result().numpy())
            auc.reset_states()

            losses = tf.Variable(0.)
            with strategy.scope():
                for imgs, lbls in ds_valid_dist:
                    losses.assign_add(validatation_dist(imgs, lbls))
            print('loss_validation: ',(losses / (VAL_SIZE // BATCH_SIZE)).numpy())     
            print('AUC_validation: ', auc.result().numpy())
            if auc.result().numpy() > validation_res:
                print('model saved!')
                model.save_weights(f'saved_model_{seed}_{fold}.h5')
                validation_res = auc.result().numpy()
            if (epoch > 5) and (epoch % 4 == 0):
                print(f'learning rate: {opt.learning_rate * 0.1}')
                model.load_weights(f'saved_model_{seed}_{fold}.h5')
                opt.learning_rate.assign(opt.learning_rate * 0.1)
            auc.reset_states()
    #         if epoch > 10:
    #             opt.learning_rate

27025408/27018416 [==============================] - 0s 0us/step


  0%|          | 0/313 [00:00<?, ?it/s]

Epoch 0 Fold 0:


 94%|█████████▎| 293/313 [03:27<00:14,  1.41it/s]


loss_training:  0.027586866
AUC_training:  0.66144615
loss_validation:  0.025889073
AUC_validation:  0.8231845
model saved!


...

loss_training:  0.003845826
AUC_training:  0.98740584


  0%|          | 0/313 [00:00<?, ?it/s]

loss_validation:  0.004892474
AUC_validation:  0.97467035
Epoch 19 Fold 4:


100%|██████████| 313/313 [01:42<00:00,  3.04it/s]


loss_training:  0.003921284
AUC_training:  0.9853397
loss_validation:  0.004876547
AUC_validation:  0.9751917


In [9]:
# train_tfrec = np.array(tf.io.gfile.glob(GCS_PATH_train_p1 + '/train_tfrec/*') + 
#                        tf.io.gfile.glob(GCS_PATH_train_p2 + '/train_tfrec/*') +
#                        tf.io.gfile.glob(GCS_PATH_train_p3 + '/train_tfrec/*') +
#                        tf.io.gfile.glob(GCS_PATH_train_p4 + '/train_tfrec/*'))
# test_OOF = train_tfrec[0]

# test_files = tf.io.gfile.glob(GCS_PATH_test_p1 + '/test_tfrec/*') +\
#              tf.io.gfile.glob(GCS_PATH_test_p2 + '/test_tfrec/*') +\
#              tf.io.gfile.glob(GCS_PATH_test_p3 + '/test_tfrec/*') +\
#              tf.io.gfile.glob(GCS_PATH_test_p4 + '/test_tfrec/*')

# ds_test = tf.data.TFRecordDataset(test_files, num_parallel_reads=AUTO)
# ds_test = ds_test.map(read_tfrec_test, num_parallel_calls=AUTO)
# ds_test = ds_test.map(lambda img, lbl: (tf.transpose(prepare_img(img), [2, 0, 1]), lbl))
# ds_test = ds_test.batch(BATCH_SIZE * 8).prefetch(AUTO)
# ds_test_dist = strategy.experimental_distribute_dataset(ds_test)


# ds_OOF = tf.data.TFRecordDataset([test_OOF], num_parallel_reads=AUTO)
# ds_OOF = ds_OOF.map(read_tfrec, num_parallel_calls=AUTO)
# ds_OOF = ds_OOF.map(lambda img, lbl: (tf.transpose(prepare_img(img), [2, 0, 1]), lbl))
# ds_OOF = ds_OOF.batch(BATCH_SIZE * 8).prefetch(AUTO)
# ds_OOF_dist = strategy.experimental_distribute_dataset(ds_OOF)

# with strategy.scope():
#     model = create_model()

# def test_(imgs):
#     return model(imgs)

# @tf.function
# def test_dist(imgs):
#     per_rep_pred = strategy.run(test_, args=(imgs,))
#     return per_rep_pred

# for seed in range(0, 1):
#     for fold in range(5):
#         model.load_weights(f'../input/seti-soheil-norm/saved_model_{seed}_{fold}.h5')
#         with strategy.scope():
#             for i, (imgs, name) in enumerate(ds_test_dist):
#                 if i % 5 == 0:
#                     print(i)
#                 _pred = tf.cast(tf.concat(test_dist(imgs).values, axis=0), tf.float32)
#                 _name = tf.concat(name.values, axis=0)[:, None]
#                 pred_ = _pred if i == 0 else tf.concat((pred_, _pred), axis=0)
#                 name_ = _name if i == 0 else tf.concat((name_, _name), axis=0)
#             all_preds = pred_ if fold == 0 else tf.concat((all_preds, pred_), axis=1)
#             for i, (imgs, target) in enumerate(ds_OOF_dist):
#                 if i % 5 == 0:
#                     print(i)
#                 _pred = tf.cast(tf.concat(test_dist(imgs).values, axis=0), tf.float32)
#                 OOF_preds = _pred if i == 0 else tf.concat((OOF_preds, _pred), axis=0)
#                 target_ = tf.concat(target.values, axis=0)[:, None]
#                 target__ = target_ if i == 0 else tf.concat((target__, target_), axis=0)
#             all_OOF_preds = OOF_preds if fold == 0 else tf.concat((all_OOF_preds, OOF_preds), axis=1)

In [10]:
# from sklearn.metrics import auc, roc_curve
# auc_score = []
# for i in [0, 3, 4]:
#     fpr, tpr, thresholds = roc_curve(target__, all_OOF_preds[:, i][:, None], pos_label=1)
#     plt.plot(fpr, tpr)
#     plt.show()
#     print(auc(fpr, tpr))
#     auc_score.append(auc(fpr, tpr))

In [11]:
# sc_ = [sc / sum(auc_score) for sc in auc_score]
# predss = (all_preds.numpy()[:, [0, 3, 4]] * np.array(sc_) + [-0.1, 0.19, -0.09]).sum(axis=1)[:, None]
# pred_df = pd.DataFrame(np.concatenate((name_.numpy(), predss), axis=1), columns=['id', 'target'])
# pred_df['id'] = pred_df['id'].apply(lambda x: x.decode())
# pred_df.sort_values('id').reset_index(drop=True).to_csv("sub5.csv", index=False)